# Epoching + TFRs for all subjects


*Created: 09/14/2024* \
*Updated: 11/20/2024* 

- epoched and baselined all patients data together - CPE + Decision Epochs
- no TFR plotting

In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
from ast import literal_eval



import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/'
anat_dir   = f'{base_dir}recon_labels/'
neural_dir = f'{base_dir}data/'
behav_dir  = f'{base_dir}behav/behav_data/'
save_path  = f'{base_dir}epoch_data/'
pt_path    = f'{base_dir}behav/subj_pt_data/'
os.makedirs(save_path,exist_ok=True)

date = datetime.date.today().strftime('%m%d%Y')
print(date)

11262024


In [4]:
import sys
sys.path.append(f'{base_dir}LFPAnalysis/')

In [5]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

In [6]:
# sys.path.append(f'{behav_dir}scripts/')
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'
sys.path.append(f'{script_dir}behav/')

from behav_utils import *
from swb_subj_behav import * 

In [12]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))


numpy==1.26.4
mne==1.8.0
seaborn==0.13.2
pandas==2.2.2
joblib==1.3.1
re==2.2.1
scipy==1.14.1


In [13]:
import sys
print(sys.version)

3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:43:22) [GCC 12.3.0]


## Gather All Subj Info

In [62]:
# Subj list for CCN 2024
subj_ids = list(pd.read_excel(f'{base_dir}subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
subj_ids


['MS002',
 'MS003',
 'MS009',
 'MS011',
 'MS015',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS024',
 'MS025',
 'MS026',
 'MS027',
 'MS028',
 'MS029',
 'MS030',
 'MS033',
 'MS035',
 'MS041',
 'MS043',
 'MS048',
 'MS050',
 'DA8',
 'DA023',
 'DA026',
 'DA037',
 'DA039']

In [63]:
anat_info_all_subj = {f'{subj_id}': pd.read_csv(f'{anat_dir}{subj_id}_labels.csv') for subj_id in subj_ids}


In [64]:
anat_info_all_subj

{'MS002':        label BN246label          x          y          z      mni_x  \
 0     LaCaS1    A32sg_L  -6.784557  49.555827   0.149337  -6.382462   
 1    LaCaS10      A9l_L  -8.380923  61.155821  43.676991  -8.220073   
 2    LaCaS11      A9l_L  -8.780014  62.355821  48.469026  -8.542609   
 3    LaCaS12    Unknown  -8.780014  63.555820  53.261062  -8.436667   
 4     LaCaS2    A32sg_L  -6.784557  50.755827   4.941372  -6.368174   
 ..       ...        ...        ...        ...        ...        ...   
 153   RmOIF5  A12/47o_R  33.922762  42.355831  -8.636061  34.666463   
 154   RmOIF6  A12/47l_R  37.913675  41.555831  -6.240044  38.716568   
 155   RmOIF7  A12/47l_R  41.904589  40.755832  -3.844026  42.794042   
 156   RmOIF8  A12/47l_R  46.294594  39.555832  -1.048672  47.308597   
 157   RmOIF9     A45r_R  50.285508  38.755833   1.746682  51.450368   
 
          mni_y      mni_z       gm  \
 0    37.158688  -3.130044     Gray   
 1    48.757099  44.868611     Gray   
 2    49

In [65]:
anat_info_all_subj.keys()

dict_keys(['MS002', 'MS003', 'MS009', 'MS011', 'MS015', 'MS016', 'MS017', 'MS019', 'MS022', 'MS024', 'MS025', 'MS026', 'MS027', 'MS028', 'MS029', 'MS030', 'MS033', 'MS035', 'MS041', 'MS043', 'MS048', 'MS050', 'DA8', 'DA023', 'DA026', 'DA037', 'DA039'])

# Load Behav Files 

In [72]:
# load task_dfs into master list 
raw_behav = [pd.read_csv(f'{behav_dir}{subj_id}_task_df.csv') for subj_id in subj_ids]
all_behav,beh_drops = format_all_behav(raw_behav, return_drops=True, drop_bads=False, drop_bads_t1=False,
                     norm=True,norm_type='zscore',pt_dir=pt_path)
all_behav

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,Choice_Util_raw,Choice_Util_t1_raw,risk_raw,risk_t1_raw,loss_raw,loss_t1_raw,Unnamed: 0_raw,Unnamed: 0_t1_raw,P_Safe_raw,P_Safe_t1_raw
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,0.301649,-1.159835,0.819173,0.819173,2.373595,2.373595,0,1.0,0.897969,0.638282
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,-1.165390,1.285700,0.819173,0.819173,2.373595,2.373595,1,2.0,0.638282,0.341225
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,1.287589,0.302752,0.819173,0.819173,2.373595,2.373595,2,3.0,0.341225,0.714422
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,0.301649,1.069216,0.819173,0.819173,2.373595,2.373595,3,4.0,0.714422,0.483386
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,1.070446,0.302752,0.819173,0.819173,2.373595,2.373595,4,5.0,0.483386,0.882421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,1.703533,-0.190891,1.187095,1.187095,0.864195,0.864195,145,146.0,0.209864,0.723269
4046,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,-0.190203,-0.190891,1.187095,1.187095,0.864195,0.864195,146,147.0,0.723269,0.509159
4047,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,-0.190203,-0.190891,1.187095,1.187095,0.864195,0.864195,147,148.0,0.509159,0.585308
4048,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,-0.190203,-2.805472,1.187095,1.187095,0.864195,0.864195,148,149.0,0.585308,0.884229


In [26]:
new_cpe_epoch_id = 'CPE_Epoch'

In [27]:
# duplicating CPE epoch times to save under a different name - getting longer epoch for plotting 
behav_df_dict = {f'{subj_id}':pd.read_csv(f'{behav_dir}{subj_id}_task_df.csv') for subj_id in subj_ids}

for subj_id in subj_ids:
    df = behav_df_dict[subj_id]
    df[new_cpe_epoch_id] = df['CpeOnset'].copy()
    behav_df_dict[subj_id] = df
    # print(df.head())

In [28]:
behav_df_dict

{'MS002':     subj_id  bdi bdi_thresh  Round  TrialNum        RT   TrialOnset  \
 0     MS002   14        low      1      25.0  2.059852   513.380590   
 1     MS002   14        low      2     117.0  1.954564   522.640856   
 2     MS002   14        low      3      79.0  1.583462   531.174799   
 3     MS002   14        low      4      42.0  2.491611   545.592613   
 4     MS002   14        low      5      85.0  1.768936   555.337336   
 ..      ...  ...        ...    ...       ...       ...          ...   
 145   MS002   14        low    146     135.0  4.525226  2082.720686   
 146   MS002   14        low    147       4.0  2.020671  2094.678701   
 147   MS002   14        low    148     136.0  2.724080  2107.766044   
 148   MS002   14        low    149      66.0  1.595996  2117.339199   
 149   MS002   14        low    150      22.0  1.854358  2126.290743   
 
      ChoiceOnset  DecisionOnset  FeedbackOnset  ... TrialEV_t1  CR_t1  \
 0     513.390239     515.450091     515.457173  ..

# Create and Save Epoched Data for all subjects

In [48]:
# IED removal requires that we set some parameters for IED detection. 
# 1. peak_thresh: how many stds should they exceed the baseline by? 
# 2. closeness_thresh: how close should they be allowed to be (in sec) to other candidate IEDs? 
# 3. width_thresh: how wide should they have to be (in sec)?

# Defaults:
IED_args = {'peak_thresh':5,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}

In [49]:
# Define spectrogram parameters for morlet wavelets 

freqs = np.logspace(*np.log10([2, 200]), num=30)
n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))


In [50]:
freqs

array([  2.        ,   2.3442046 ,   2.74764759,   3.22052406,
         3.77478364,   4.42443258,   5.18588759,   6.07839076,
         7.12449578,   8.35063787,   9.78780184,  11.47230502,
        13.44671507,  15.76092563,  18.47341714,  21.65273468,
        25.37922006,  29.74704215,  34.86657644,  40.86719436,
        47.9005324 ,  56.14432408,  65.80689125,  77.13240842,
        90.40707313, 105.96633813, 124.20338838, 145.57907688,
       170.63357048, 200.        ])

In [51]:
#### Set global vars for all subj 

baseline_ev  = 'TrialOnset' # fixation cross baseline epoch
analysis_evs = ['DecisionOnset','CpeOnset','CPE_Epoch']

# dictionary of event ids and times - repeated from above
evs = {'TrialOnset': [-1.0, 0], #ITI is always at least one second + 10ms buffer for wavelet padding (no horizontal smearing)
       'DecisionOnset': [-0.5, 0], # 0.5 sec prior to choice
       'CpeOnset': [0.0,3.0], # 3 sec following CPE outcome reveal - timestamps are made manually above
       'CPE_Epoch': [-0.5,2.5] } # 3 sec following CPE outcome reveal - timestamps are made manually above



In [52]:

for subj_id in subj_ids: 
    
    print(f'start epoching {subj_id}')
    
    # load photodiode data
    diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
    slope,offset = np.load(diode_info_fname)
    
    # get subj behav data --- make sure to check that all of these are the same number of trials 
    behav_df = behav_df_dict[subj_id]    

    # make dictionary to store epoch data 
    epochs_all_evs = {f'{x}': np.nan for x in evs}
    
    for event in evs.keys():
        ### make epochs should now find IEDs, artifact times, and save info as df in load_path directory for every ev
        # Make the epochs. 
        ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
                                                     slope=slope, offset=offset,
                                                     behav_name=event, behav_times=behav_df[event].values,
                                                     ev_start_s=evs[event][0], ev_end_s=evs[event][1],
                                                     buf_s = 1, IED_args=IED_args,
                                                     downsamp_factor=None,  
                                                     detrend=0, 
                                                     #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html
                                                     save_path = save_path, ### SWB PIPELINE ADDITION - ADD SAVE PATH & SUBJ ID FOR DATA ORGANIZATION
                                                     subj_id=subj_id)

        epochs_all_evs[event] = ev_epochs
        
        ### save epoched data - make new data saving path
        ev_path = f'{save_path}{event}/'
        os.makedirs(ev_path,exist_ok=True)
        ### save baseline tfr data         
        ev_epochs.save(f'{ev_path}/{subj_id}_{event}_epochs.fif', overwrite=True)    
        
        del ev_epochs 
    
    #### COMPUTE BASELINE TFRs FOR ONE SUBJ
    
    # baseline epoch dictionary 
    epochs_all_baseline = {} 

    # baseline event = TrialOnset epochs 
    baseline_epochs_reref = epochs_all_evs[baseline_ev]

    # dropping bad channels from baseline epochs 
    baseline_epochs_reref = baseline_epochs_reref.drop_channels(baseline_epochs_reref.info['bads'])

    # compute baseline tfr 
    temp_pow  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
                                              picks=baseline_epochs_reref.ch_names, # now doesn't include bad ch
                                              use_fft=True, n_jobs=-1, output='power',
                                              return_itc=False, average=False) # returns pow (not ITC) for all epochs

    temp_pow.crop(tmin=evs[baseline_ev][0], tmax=evs[baseline_ev][1]) # clipping epoch to remove buffer

    ### DROP IED/Artifacts from baseline! Remove bad data by naning out +/-100ms around bad timepoints (IEDs and artifacts)
    
    # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
    IED_df      = pd.read_csv(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}_IED_df.csv') 
    artifact_df = pd.read_csv(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}_artifact_df.csv') 

    # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
    for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names):
        
        # lists of IED/artifact timestamps from dfs for one channel (if any)
        ied_ev_list = IED_df[ch_name].dropna().index.tolist()
        artifact_ev_list = artifact_df[ch_name].dropna().index.tolist()

        # iterate through every IED timestamp in channel 
        for ev_ in ied_ev_list: 
            # iterate through values in IED df (literal eval = evaluates string as literal python data structures)
            for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
                # remove 100 ms before 
                ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                # remove 100 ms after 
                ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

        # iterate through every artifact timestamp in channel 
        for ev_ in artifact_ev_list: 
             # iterate through values in artifact df (literal eval = evaluates string as literal python data structures)
            for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
                # remove 100 ms before 
                ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                # remove 100 ms after
                ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

    #### save baseline tfr
    temp_pow.save(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}-tfr.h5', overwrite=True) 
    #### store baseline tfr data to dict
    epochs_all_baseline[baseline_ev] = temp_pow
    
    # delete clunky data (could rename temp_pow instead of double saving because there's only one baseline)
    del temp_pow,IED_df,artifact_df,baseline_epochs_reref 
    

    ##### COMPUTE TFR FOR ANALYSIS EPOCHS AND BASELINE TO BASELINE TFR DATA
        # repeat baselining procedure for every event in analysis_evs
        # remove +/-100ms around IED/artifact individually for each analysis epoch 
        # baseline analysis ev tfrs
        #### IGNORE BELOW FOR NOW
            # iteratively check for zscores still above 10 std, remove those timepoints, then recompute baselined tfr


    for event in analysis_evs: 
        
        print('Compute baselined TFR for epoch: ',event)
        
        # extract raw epoch data for analysis ev 
        event_epochs_reref = epochs_all_evs[event].copy()
        # dropping bad channels from baseline epochs --- don't think there's any bad ch in fif
        event_epochs_reref = event_epochs_reref.drop_channels(event_epochs_reref.info['bads'])
        ch_names = event_epochs_reref.ch_names

        # compute raw event TFR from raw epoched data
        temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
                                                 picks=event_epochs_reref.ch_names, 
                                                 use_fft=True, n_jobs=-1, output='power', 
                                                 return_itc=False, average=False)
        # crop 1.0 sec buffer from analysis epoch 
        temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

        #### DROP IEDs/ARTIFACTS FROM EVENT EPOCH repeat IED/artifact removal procedure used for baseline epochs, but with analysis event IED/artifact times

        # load IED/artifact dfs specific to event in analysis evs
        IED_df      = pd.read_csv(f'{save_path}{event}/{subj_id}_{event}_IED_df.csv') 
        artifact_df = pd.read_csv(f'{save_path}{event}/{subj_id}_{event}_artifact_df.csv') 

        # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
        for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
            ied_ev_list = IED_df[ch_name].dropna().index.tolist()
            artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
            for ev_ in ied_ev_list: 
                for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
                    # remove 100 ms before 
                    ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    # remove 100 ms after
                    ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
            for ev_ in artifact_ev_list: 
                for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
                    # remove 100 ms before 
                    ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    # remove 100 ms after
                    ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

        ### COMPUTE NORMALIZED TFR FOR EVENT EPOCH
        baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data,
                                                                       include_epoch_in_baseline=False,  
                                                                       baseline_mne=epochs_all_baseline[baseline_ev].data,  
                                                                       mode='zscore', ev_axis=0, elec_axis=1, 
                                                                       freq_axis=2, time_axis=3)

        # ############ TEST 11/20/24 COMPARE DATA WITH/WITHOUT ABSURDITY THRESHOLDING ############
        
        # #### remove extreme power values (z>10) remaining after IED/artifact removal 
        # # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters

        # # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
        # absurdity_threshold = 10 # zscore threshold for removal
        # max_iter = 10 # max number of iterations for while loop 
        # large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

        # iteration = 0 # intialize iter counter
        # while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
        #     print(f'baseline z-score iteration # {iteration}')
        #     # Baseline by all the baseline periods in the session
        #     baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data,include_epoch_in_baseline=False, 
        #                                                                            baseline_mne=epochs_all_baseline[baseline_ev].data,  
        #                                                                            mode='zscore', ev_axis=0, elec_axis=1, 
        #                                                                            freq_axis=2, time_axis=3)

        #     large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
        #     if large_z_mask[0].shape[0] == 0:
        #         # no more large z
        #         large_z_flag = False
        #     else:
        #         # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
        #         # contamination of all z's
        #         temp_pow.data[large_z_mask] = np.nan

        #     iteration +=1

        # input baseline corrected pow into mne EpochsTFR object once all z>10 removed or iteratively removed 10x 
        
        zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power,
                                            temp_pow.times, freqs)

        # add metadata to baseline corrected TFR event epochs object 
#         zpow.metadata = event_epochs_reref.metadata

        # save data (#conda install -c conda-forge h5io must install to save tfr data)
        zpow.save(f'{save_path}{event}/{subj_id}_{event}_noabsurd-tfr.h5', overwrite=True) 

        print(f'finished epoching {subj_id}')
        
        del zpow, temp_pow, event_epochs_reref
    
    del epochs_all_evs, epochs_all_baseline



start epoching MS002
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 265 samples (0.530 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    4.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.5s finished


Used Annotations descriptions: ['TrialOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  89 out of  96 | elapsed:    2.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.2s finished


Used Annotations descriptions: ['DecisionOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1251 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 

...

finished epoching DA039
Compute baselined TFR for epoch:  CpeOnset
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   13.8s finished


finished epoching DA039
Compute baselined TFR for epoch:  CPE_Epoch
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   13.8s finished


finished epoching DA039


In [53]:

for subj_id in subj_ids: 
    
    print(f'start epoching {subj_id}')
    
    # load photodiode data
    diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
    slope,offset = np.load(diode_info_fname)
    # get subj behav data --- make sure to check that all of these are the same number of trials 
    behav_df = behav_df_dict[subj_id]    

    # make dictionary to store epoch data 
    epochs_all_evs = {f'{x}': np.nan for x in evs}
    
    for event in evs.keys():
        ### make epochs should now find IEDs, artifact times, and save info as df in load_path directory for every ev
        # Make the epochs. 
        ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
                                                     slope=slope, offset=offset,
                                                     behav_name=event, behav_times=behav_df[event].values,
                                                     ev_start_s=evs[event][0], ev_end_s=evs[event][1],
                                                     buf_s = 1, IED_args=IED_args,
                                                     downsamp_factor=None,  
                                                     detrend=0, 
                                                     #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html
                                                     save_path = save_path, ### SWB PIPELINE ADDITION - ADD SAVE PATH & SUBJ ID FOR DATA ORGANIZATION
                                                     subj_id=subj_id)

        epochs_all_evs[event] = ev_epochs
        
        ### save epoched data - make new data saving path
        ev_path = f'{save_path}{event}/'
        os.makedirs(ev_path,exist_ok=True)
        
        ### save baseline tfr data         
        ev_epochs.save(f'{ev_path}/{subj_id}_{event}_epochs.fif', overwrite=True)    
        del ev_epochs 
        
    #### COMPUTE BASELINE TFRs FOR ONE SUBJ
    
    # baseline epoch dictionary 
    epochs_all_baseline = {} 

    # baseline event = TrialOnset epochs 
    baseline_epochs_reref = epochs_all_evs[baseline_ev]

    # dropping bad channels from baseline epochs 
    baseline_epochs_reref = baseline_epochs_reref.drop_channels(baseline_epochs_reref.info['bads'])

    # compute baseline tfr 
    temp_pow  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
                                              picks=baseline_epochs_reref.ch_names, # now doesn't include bad ch
                                              use_fft=True, n_jobs=-1, output='power',
                                              return_itc=False, average=False) # returns pow (not ITC) for all epochs

    temp_pow.crop(tmin=evs[baseline_ev][0], tmax=evs[baseline_ev][1]) # clipping epoch to remove buffer

    ### DROP IED/Artifacts from baseline! Remove bad data by naning out +/-100ms around bad timepoints (IEDs and artifacts)
    
    # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
    IED_df      = pd.read_csv(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}_IED_df.csv') 
    artifact_df = pd.read_csv(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}_artifact_df.csv') 

    # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
    for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names):
        
        # lists of IED/artifact timestamps from dfs for one channel (if any)
        ied_ev_list = IED_df[ch_name].dropna().index.tolist()
        artifact_ev_list = artifact_df[ch_name].dropna().index.tolist()

        # iterate through every IED timestamp in channel 
        for ev_ in ied_ev_list: 
            # iterate through values in IED df (literal eval = evaluates string as literal python data structures)
            for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
                # remove 100 ms before 
                ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                # remove 100 ms after 
                ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

        # iterate through every artifact timestamp in channel 
        for ev_ in artifact_ev_list: 
             # iterate through values in artifact df (literal eval = evaluates string as literal python data structures)
            for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
                # remove 100 ms before 
                ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                # remove 100 ms after
                ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

    #### save baseline tfr
    temp_pow.save(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}-tfr.h5', overwrite=True) 
    #### store baseline tfr data to dict
    epochs_all_baseline[baseline_ev] = temp_pow
    
    # delete clunky data (could rename temp_pow instead of double saving because there's only one baseline)
    del temp_pow,IED_df,artifact_df,baseline_epochs_reref 
    

    ##### COMPUTE TFR FOR ANALYSIS EPOCHS AND BASELINE TO BASELINE TFR DATA
        # repeat baselining procedure for every event in analysis_evs
        # remove +/-100ms around IED/artifact individually for each analysis epoch 
        # baseline analysis ev tfrs
        #### IGNORE BELOW FOR NOW
            # iteratively check for zscores still above 10 std, remove those timepoints, then recompute baselined tfr

    for event in analysis_evs: 
        
        print('Compute baselined TFR for epoch: ',event)
        
        # extract raw epoch data for analysis ev 
        event_epochs_reref = epochs_all_evs[event].copy()
        # dropping bad channels from baseline epochs --- don't think there's any bad ch in fif
        event_epochs_reref = event_epochs_reref.drop_channels(event_epochs_reref.info['bads'])
        ch_names = event_epochs_reref.ch_names

        # compute raw event TFR from raw epoched data
        temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
                                                 picks=event_epochs_reref.ch_names, 
                                                 use_fft=True, n_jobs=-1, output='power', 
                                                 return_itc=False, average=False)
        # crop 1.0 sec buffer from analysis epoch 
        temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

        #### DROP IEDs/ARTIFACTS FROM EVENT EPOCH repeat IED/artifact removal procedure used for baseline epochs, but with analysis event IED/artifact times

        # load IED/artifact dfs specific to event in analysis evs
        IED_df      = pd.read_csv(f'{save_path}{event}/{subj_id}_{event}_IED_df.csv') 
        artifact_df = pd.read_csv(f'{save_path}{event}/{subj_id}_{event}_artifact_df.csv') 

        # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
        for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
            ied_ev_list = IED_df[ch_name].dropna().index.tolist()
            artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
            for ev_ in ied_ev_list: 
                for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
                    # remove 100 ms before 
                    ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    # remove 100 ms after
                    ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
            for ev_ in artifact_ev_list: 
                for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
                    # remove 100 ms before 
                    ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    # remove 100 ms after
                    ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

        
        ### COMPUTE NORMALIZED TFR FOR EVENT EPOCH
        baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data,
                                                                       include_epoch_in_baseline=False,  
                                                                       baseline_mne=epochs_all_baseline[baseline_ev].data,  
                                                                       mode='zscore', ev_axis=0, elec_axis=1, 
                                                                       freq_axis=2, time_axis=3)

                
        #### remove extreme power values (z>10) remaining after IED/artifact removal 
        # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters

        # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
        absurdity_threshold = 10 # zscore threshold for removal
        max_iter = 10 # max number of iterations for while loop 
        large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

        iteration = 0 # intialize iter counter
        while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
            print(f'baseline z-score iteration # {iteration}')
            # Baseline by all the baseline periods in the session
            baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data,include_epoch_in_baseline=False, 
                                                                                   baseline_mne=epochs_all_baseline[baseline_ev].data,  
                                                                                   mode='zscore', ev_axis=0, elec_axis=1, 
                                                                                   freq_axis=2, time_axis=3)

            large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
            if large_z_mask[0].shape[0] == 0:
                # no more large z
                large_z_flag = False
            else:
                # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
                # contamination of all z's
                temp_pow.data[large_z_mask] = np.nan

            iteration +=1

        # input baseline corrected pow into mne EpochsTFR object once all z>10 removed or iteratively removed 10x 
        zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power,
                                            temp_pow.times, freqs)

        # save data (#conda install -c conda-forge h5io must install to save tfr data)
        zpow.save(f'{save_path}{event}/{subj_id}_{event}-tfr.h5', overwrite=True) 
        
        print(f'finished epoching {subj_id}')
        
        del zpow, temp_pow, event_epochs_reref
    
    del epochs_all_evs, epochs_all_baseline


start epoching MS002
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 265 samples (0.530 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  87 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    3.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.5s finished


Used Annotations descriptions: ['TrialOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1501 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  89 out of  96 | elapsed:    2.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.1s finished


Used Annotations descriptions: ['DecisionOnset']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 1251 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transiti

...

baseline z-score iteration # 0
baseline z-score iteration # 1
finished epoching DA039
Compute baselined TFR for epoch:  CpeOnset
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   13.6s finished


baseline z-score iteration # 0
baseline z-score iteration # 1
finished epoching DA039
Compute baselined TFR for epoch:  CPE_Epoch
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:   13.5s finished


baseline z-score iteration # 0
baseline z-score iteration # 1
finished epoching DA039


In [ ]:
        # #### remove extreme power values (z>10) remaining after IED/artifact removal 
        # # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters

        # # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
        # absurdity_threshold = 10 # zscore threshold for removal
        # max_iter = 10 # max number of iterations for while loop 
        # large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

        # iteration = 0 # intialize iter counter
        # while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
        #     print(f'baseline z-score iteration # {iteration}')
        #     # Baseline by all the baseline periods in the session
        #     baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, 
        #                           baseline_mne=epochs_all_baseline[baseline_ev].data,  
        #                           mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

        #     large_z_mask = np.where(baseline_corrected_power>absurdity_threshold)
        #     if large_z_mask[0].shape[0] == 0:
        #         # no more large z
        #         large_z_flag = False
        #     else:
        #         # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
        #         # contamination of all z's
        #         temp_pow.data[large_z_mask] = np.nan

        #     iteration +=1

In [ ]:
        # # Let's iteratively nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
        # if uncaptured_z_thresh: 
        #     absurdity_threshold = 10
        #     max_iter = 10
        #     large_z_flag=True 

        #     # iteration = 0

        #     while (large_z_flag==True) & (iteration<max_iter): 
        #         print(f'baseline z-score iteration # {iteration}')
        #         # Baseline by all the baseline periods in the session
        #         baseline_corrected_power = baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
        #                               baseline_mne=baseline_power.data, 
        #                               mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

        #         large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
        #         if large_z_mask[0].shape[0] == 0:
        #             # no more large z
        #             large_z_flag = False
        #         else:
        #             # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
        #             # contamination of all z's
        #             temp_pow.data[large_z_mask] = np.nan

        #         iteration +=1
        

In [44]:

# for subj_id in subj_ids: 
#     print(f'start epoching {subj_id}')
    
#     # load photodiode data
#     diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
#     slope,offset = np.load(diode_info_fname)
#     # load behav data 
#     behav_df = pd.read_csv(f'{behav_dir}{subj_id}_task_df.csv')

#     epochs_all_evs = {f'{x}': np.nan for x in evs}
    
#     ### make epochs should now find IEDs, artifact times, and save info as df in load_path directory for every ev

#     for event in evs.keys():
#         # Make the epochs. 
#         ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
#                                                      slope=slope, offset=offset,
#                                                      behav_name=event, behav_times=behav_df[event].values,
#                                                      ev_start_s=evs[event][0], ev_end_s=evs[event][1],
#                                                      buf_s = 1, IED_args=IED_args,
#                                                      downsamp_factor=None,  
#                                                      detrend=0, #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html
#                                                      save_path = save_path,subj_id=subj_id)

#         epochs_all_evs[event] = ev_epochs
#         ### save epoched data 
#         ev_epochs.save(f'{neural_dir}{subj_id}/{subj_id}_{event}_epochs.fif', overwrite=True)
    
    
#     #### COMPUTE BASELINE TFRs FOR ONE SUBJ
#     # baseline epoch dictionary 
#     epochs_all_baseline = {} 

#     # baseline event = TrialOnset epochs 
#     baseline_epochs_reref = epochs_all_evs[baseline_ev]

#     # dropping bad channels from baseline epochs 
#     baseline_epochs_reref = baseline_epochs_reref.drop_channels(baseline_epochs_reref.info['bads'])

#     # compute baseline tfr 
#     temp_pow  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
#                                               picks=baseline_epochs_reref.ch_names, # now doesn't include bad ch
#                                               use_fft=True, n_jobs=-1, output='power',
#                                               return_itc=False, average=False) # returns pow (not ITC) for all epochs

#     temp_pow.crop(tmin=evs[baseline_ev][0], tmax=evs[baseline_ev][1]) # clipping epoch to remove buffer

#     # Remove bad data by naning out +/-100ms around bad timepoints (IEDs and artifacts)
#     # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#     IED_df = pd.read_csv(f'{neural_dir}{subj_id}/{baseline_ev}_IED_df.csv') 
#     artifact_df = pd.read_csv(f'{neural_dir}{subj_id}/{baseline_ev}_artifact_df.csv') 

#     # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
#     for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names):
#         # lists of IED/artifact timestamps from dfs for one channel (if any)
#         ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#         artifact_ev_list = artifact_df[ch_name].dropna().index.tolist()

#         # iterate through every IED timestamp in channel 
#         for ev_ in ied_ev_list: 
#             # iterate through values in IED df (literal eval = evaluates string as literal python data structures)
#             for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                 # remove 100 ms before 
#                 ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 # remove 100 ms after 
#                 ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

#         # iterate through every artifact timestamp in channel 
#         for ev_ in artifact_ev_list: 
#              # iterate through values in artifact df (literal eval = evaluates string as literal python data structures)
#             for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                 # remove 100 ms before 
#                 ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 # remove 100 ms after
#                 ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

#     epochs_all_baseline[baseline_ev] = temp_pow


#     ##### COMPUTE TFR FOR ANALYSIS EPOCHS AND BASELINE TO BASELINE TFR DATA

#         # repeat baselining procedure for every event in analysis_evs
#         # remove +/-100ms around IED/artifact individually for each analysis epoch 
#         # baseline analysis ev tfrs
#         # iteratively check for zscores still above 10 std, remove those timepoints, then recompute baselined tfr


#     power_epochs = {} # keys = analysis evs 

#     for event in analysis_evs: 
#         print('Compute baselined TFR for epoch: ',event)

#         # extract raw epoch data for analysis ev 
#         event_epochs_reref = epochs_all_evs[event].copy()
#         # dropping bad channels from baseline epochs 
#         event_epochs_reref = event_epochs_reref.drop_channels(event_epochs_reref.info['bads'])
#         ch_names = event_epochs_reref.ch_names

#         # compute TFR from raw epoched data
#         temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
#                                                  picks=event_epochs_reref.ch_names, 
#                                                  use_fft=True, n_jobs=-1, output='power', 
#                                                  return_itc=False, average=False)
#         # crop 1.0 sec buffer from analysis epoch 
#         temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

#         ### repeat IED/artifact removal procedure used for baseline epochs, but with analysis event IED/artifact times

#         # load IED/artifact dfs specific to event in analysis evs
#         IED_df = pd.read_csv(f'{neural_dir}{subj_id}/{event}_IED_df.csv') 
#         artifact_df = pd.read_csv(f'{neural_dir}{subj_id}/{event}_artifact_df.csv') 

#         # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
#         for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
#             ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#             artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#             for ev_ in ied_ev_list: 
#                 for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#             for ev_ in artifact_ev_list: 
#                 for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

#         ##### remove extreme power values (z>10) remaining after IED/artifact removal 
#         # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters


#         # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
#         absurdity_threshold = 10 # zscore threshold for removal
#         max_iter = 10 # max number of iterations for while loop 
#         large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

#         iteration = 0 # intialize iter counter
#         while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
#             print(f'baseline z-score iteration # {iteration}')
#             # Baseline by all the baseline periods in the session
#             baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, 
#                                   baseline_mne=epochs_all_baseline[baseline_ev].data,  
#                                   mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

#             large_z_mask = np.where(baseline_corrected_power>absurdity_threshold)
#             if large_z_mask[0].shape[0] == 0:
#                 # no more large z
#                 large_z_flag = False
#             else:
#                 # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
#                 # contamination of all z's
#                 temp_pow.data[large_z_mask] = np.nan

#             iteration +=1

#         # input baseline corrected pow into mne EpochsTFR object once all z>10 removed or iteratively removed 10x 
#         zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power,
#                                             temp_pow.times, freqs)

#         # add metadata to baseline corrected TFR event epochs object 
# #         zpow.metadata = event_epochs_reref.metadata

#         # save data (#conda install -c conda-forge h5io must install to save tfr data)
#         zpow.save(f'{neural_dir}{subj_id}/{subj_id}_{event}-tfr.h5', overwrite=True) 

#         # store event zpow epochs in dict for all analysis evs 
#         power_epochs[event] = zpow
        
#         print(f'finished epoching {subj_id}')
        
#     del power_epochs, zpow, temp_pow, large_z_mask, event_epochs_reref, ev_epochs, epochs_all_evs, epochs_all_baseline






# Plot TFRs 
- For each analysis event, plot the mean TFR across all epochs for all channels
- Save plots
- Visually inspect TFRs to see if further data cleaning is necessary

In [ ]:
# ####add directory for TFRs
# tfr_dir = f'{base_dir}ephys_analysis/figs/single_elec_tfrs/{subj_id}/'

# # print date of TFR plotting 
# date = datetime.date.today().strftime('%m%d%Y')
# print(date, offset) # print date, double check current slope/offset


In [ ]:
# #### for each analysis event - plot one TFR averaged across all epochs for every electrode & save 

# for event in analysis_evs: 
#     print('Plot TFR for epoch: ',event)
#     save_path = f'{tfr_dir}{event}/'
#     os.makedirs(save_path,exist_ok = True) #added so you don't have to manually make folders
    
#     ev_data = power_epochs[event].copy() # tfr plot data - epochs x num ch x freqs x times 
#     times = ev_data.times.copy() # same for all elec
    
#     for ch in ev_data.ch_names:
#         # single ch tfr 
#         plot_data = np.nanmean(np.nanmean(ev_data.copy().pick_channels([ch]).data, axis=0), axis=0) 
        
#         fig, ax = plt.subplots(1, 1, figsize=(6, 4),dpi=300)
        
#         im = ax.imshow(plot_data,interpolation='Bicubic',
#                        aspect='auto', origin='lower', cmap='RdBu_r')
    

#         ax.set(xlabel='Time (s)', ylabel='Frequency (Hz)',title=f'{subj_id}_{ch}_{event}_{date}',
#                yticks=np.arange(0,len(freqs))[::5], 
#                yticklabels=freqs[::5].astype(int), 
#                xticks = np.arange(plot_data.shape[1])[::250],
#                   xticklabels= times[::250])
        
#         fig.colorbar(im, ax=ax)

#         plt.savefig(f'{save_path}{subj_id}_{ch}_{event}_{date}.png', dpi=300)
# #         bbox_inches=None, pad_inches=0.1,
# #         facecolor='auto', edgecolor='auto',backend=None)
#         plt.close()

    

In [ ]:
# # from salman - faster code

# def compute_and_baseline_tfr(baseline_event, task_events, freqs, n_cycles, load_path, save_path,
#                             IED_artifact_thresh=True, uncaptured_z_thresh=True, output='save'):
    
#     """
    
#     """
    
    
#     baseline_name = list(baseline_event.keys())[0]
    
#     # load baseline epochs
#     baseline_epochs_reref = mne.read_epochs(f'{load_path}/{baseline_name}-epo.fif', preload=True)
    
#     # compute TFR
#     baseline_power  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, 
#                                           freqs=freqs, 
#                                           n_cycles=n_cycles, 
#                                           picks=baseline_epochs_reref.ch_names,
#                                           use_fft=True, 
#                                           n_jobs=-1, 
#                                           output='power', 
#                                           return_itc=False, 
#                                           average=False)
    

#     # Crop the data to the appropriate 
#     baseline_power.crop(tmin=baseline_event[baseline_name][0], 
#                         tmax=baseline_event[baseline_name][1])
    
    
#     if IED_artifact_thresh:
#         # NAN out the bad data
#         # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#         IED_df = pd.read_csv(f'{load_path}/{baseline_name}_IED_df.csv') 
#         artifact_df = pd.read_csv(f'{load_path}/{baseline_name}_artifact_df.csv') 


#         # Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
#         for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names): 
#             ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#             artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#             for ev_ in ied_ev_list: 
#                 for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after 
#                     ev_ix_end = np.min([baseline_power.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     baseline_power.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#             for ev_ in artifact_ev_list: 
#                 for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([baseline_power.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     baseline_power.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
    
#     # remove epochs from memory
#     del baseline_epochs_reref
    
#     # Now we will deal with the task events of interest   
    
#     # make the output path for the z-scored TFRs
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
        
#     for event in task_events.keys():
        
#         event_epochs_reref = mne.read_epochs(f'{load_path}/{event}-epo.fif', preload=True)
                    
#         temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, 
#                                                  freqs=freqs, 
#                                                  n_cycles=n_cycles,
#                                                  picks=event_epochs_reref.ch_names, 
#                                                  use_fft=True, 
#                                                  n_jobs=-1, 
#                                                  output='power', 
#                                                  return_itc=False, 
#                                                  average=False)
    
#         temp_pow.crop(tmin=task_events[event][0], tmax=task_events[event][1])
        
#         if IED_artifact_thresh:
#             # NAN out the bad data
#             # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#             IED_df = pd.read_csv(f'{load_path}/{event}_IED_df.csv') 
#             artifact_df = pd.read_csv(f'{load_path}/{event}_artifact_df.csv') 

#             # Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
#             for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
#                 ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#                 artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#                 for ev_ in ied_ev_list: 
#                     for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                         # remove 100 ms before 
#                         ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         # remove 100 ms after
#                         ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#                 for ev_ in artifact_ev_list: 
#                     for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                         # remove 100 ms before 
#                         ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         # remove 100 ms after
#                         ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan    
    
#         # Compute first pass of baseline
#         baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
#                                                                        baseline_mne=baseline_power.data, 
#                                                                        mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)
            
#         # Let's iteratively nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
#         if uncaptured_z_thresh: 
#             absurdity_threshold = 10
#             max_iter = 10
#             large_z_flag=True 

#             iteration = 0

#             while (large_z_flag==True) & (iteration<max_iter): 
#                 print(f'baseline z-score iteration # {iteration}')
#                 # Baseline by all the baseline periods in the session
#                 baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
#                                       baseline_mne=baseline_power.data, 
#                                       mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

#                 large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
#                 if large_z_mask[0].shape[0] == 0:
#                     # no more large z
#                     large_z_flag = False
#                 else:
#                     # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
#                     # contamination of all z's
#                     temp_pow.data[large_z_mask] = np.nan

#                 iteration +=1
        
#         zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power, 
#                                     temp_pow.times, freqs)

#         zpow.metadata = event_epochs_reref.metadata
        
#         if output == 'save':
#             zpow.save(f'{save_path}/{event}-tfr.h5', overwrite=True)
#         elif output == 'return': 
#             return zpow 
#         elif output == 'both':
#             zpow.save(f'{save_path}/{event}-tfr.h5', overwrite=True)
#             return zpow 
        
# 12:32
# Usage example:
# # Explicitly define a list of analysis events and the baseline event. Should correspond to the dict

# baseline_event = {'baseline_start':[0, 0.75]}
# task_events = {'gamble_start':[-1.0, 0], 
#        'feedback_start': [0, 1.5]}

# day = 'Day1'

# # Define parameters for TFR: 
# freqs = np.logspace(*np.log10([2, 200]), num=30)
# n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))

# subj_id = 'MS012'

# compute_and_baseline_tfr(baseline_event, task_events, freqs, n_cycles, 
#                          load_path=f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/{day}', 
#                          save_path=f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/scratch/TFR',
#                          IED_artifact_thresh=True, uncaptured_z_thresh=True, output='save')